# Testing

In [ ]:
import tweepy
import os
import openpyxl



client = tweepy.Client(
    consumer_key=consumer_api_key,
    consumer_secret=comsumer_api_secret_key,
    access_token=access_token,
    access_token_secret=access_token_secret
)

# Get My User ID

In [7]:
def get_my_user_id():
    """Retrieve the authenticated user's ID."""
    try:
        user = client.get_me()
        print(user)
        return user.data.id if user.data else None
    except Exception as e:
        print(f"An error occurred while retrieving user ID: {e}")
        return None
    
get_my_user_id()

Response(data=<User id=1876696076499222528 name=MIND of Pepe username=MINDofPepe>, includes={}, errors=[], meta={})


1876696076499222528

# Code

# Get Replies on POST

In [31]:
import tweepy

def get_replies_to_tweet(tweet_id, bearer_token):
    client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
    query = f'conversation_id:{tweet_id} -is:retweet'
    replies = []

    try:
        for response in tweepy.Paginator(
            client.search_recent_tweets,
            query=query,
            tweet_fields=['author_id', 'created_at', 'in_reply_to_user_id'],
            expansions='author_id',
            user_fields=['username'],
            max_results=100
        ):
            if response.data:
                users = {u['id']: u for u in response.includes['users']}
                for tweet in response.data:
                    author = users.get(tweet.author_id)
                    if author:
                        replies.append({
                            'tweet_id': tweet.id,
                            'username': author.username,
                            'created_at': tweet.created_at,
                            'text': tweet.text
                        })
    except Exception as e:
        print(f"An error occurred: {e}")

    return replies

tweet_id = "1916527181767053350" 
replies = get_replies_to_tweet(tweet_id, bearer_token)
# for reply in replies:
#     print(f"Reply ID: {reply['tweet_id']}, @{reply['username']} at {reply['created_at']}: {reply['text']}")
type(replies)

list

In [32]:
replies

[{'tweet_id': 1916880419217457440,
  'username': 'Bx23Reis',
  'created_at': datetime.datetime(2025, 4, 28, 15, 41, 20, tzinfo=datetime.timezone.utc),
  'text': '@MINDofPepe Are we launching on the 10th?'},
 {'tweet_id': 1916879887668744589,
  'username': 'b0gdy_p',
  'created_at': datetime.datetime(2025, 4, 28, 15, 39, 13, tzinfo=datetime.timezone.utc),
  'text': "@MINDofPepe Hey @MINDofPepe! 👋 We’re reaching out from Tothemoon Exchange 🚀\n\nWe’d love to connect and explore a listing opportunity with you. What's the best way to get in touch?"},
 {'tweet_id': 1916776798328701121,
  'username': 'MindOfficialp',
  'created_at': datetime.datetime(2025, 4, 28, 8, 49, 35, tzinfo=datetime.timezone.utc),
  'text': '@IntrstelarMan You are welcome to reach our support team via link below on any issues buying,staking,unstaking, swapping, withdraw, contract adding  and steps to make your purchase coin  appear in your wallet ⏬ https://t.co/D58Ze0o7je'},
 {'tweet_id': 1916776681034891446,
  'userna

# EXCEL Usernames extractions

In [24]:
import pandas as pd
from urllib.parse import urlparse, unquote

def extract_usernames_from_excel(file_path):
    df = pd.read_excel(file_path)
    
    if 'Profile URL' not in df.columns:
        raise ValueError("The Excel file must contain a 'Profile URL' column.")
    
    usernames = []
    for url in df['Profile URL']:
        if isinstance(url, str):
            parsed_url = urlparse(url)
            path = parsed_url.path
            
            if path.startswith('/search') or path == '/':
                continue
            
            segments = path.strip('/').split('/')
            if segments:
                username = unquote(segments[0])
                usernames.append(username)
    
    return usernames

excel_file = 'data/MIND.xlsx'
usernames = extract_usernames_from_excel(excel_file)
usernames

['flybridge',
 'Vikcyter_2',
 'BRICSinfo',
 'rocketonxyz',
 'Permissionless',
 'TheMoonShow',
 'seedphrase',
 'tether_to',
 'Polymarket',
 'Punk583',
 'R3ACHNTWRK',
 'RobertKennedyJr',
 'oakcurrency',
 'nftfactoryparis',
 '0xPolygonFdn',
 'Solana',
 'BSCNheadlines',
 'jokerace_io',
 'X',
 'JackPosobiec',
 'dexter_cap',
 'divine_economy',
 '1PercentBetterX',
 '0xUnicorn_',
 'deekaymotion',
 'cryptonews',
 'surgence_io',
 'legendarygainz_',
 'Stripe',
 'BackedVC',
 'OthersideMeta',
 'iamneubert',
 'wallet',
 'TrustWallet',
 'BorednHungry',
 '11x_official',
 'xAI',
 'ApeCoin',
 'ton_blockchain',
 'streamhq_xyz',
 'banterboycrypto',
 'NFCSummit',
 'Lacoste',
 'DelGroyp',
 'PayPal',
 'jfx',
 'SOCKETProtocol',
 'bear_brand96',
 'get_wildfire',
 'layer3xyz',
 'cartainc',
 'delegatedotxyz',
 'paulg',
 'IdMintThat',
 'jacqmelinek',
 'HumanEvents',
 'Porsche',
 'IamNomad',
 'HexicanHeartel',
 'JimDaily',
 'DCGco',
 'Layer3FDN',
 'p_ferragu',
 'HendrixxWeb3',
 'BochainCapital',
 'BaycCouncil',
 '

# Tweets I am mentioned

In [21]:
import tweepy

def extract_mentions(bearer_token, username, max_results=100):
    client = tweepy.Client(bearer_token=bearer_token)

    user = client.get_user(username=username)
    user_id = user.data.id

    mentions = client.get_users_mentions(id=user_id, max_results=max_results, expansions='author_id', tweet_fields='created_at')

    mention_details = []

    if mentions.data:
        author_ids = {user.id: user.username for user in mentions.includes['users']}

        for tweet in mentions.data:
            author_username = author_ids.get(tweet.author_id, 'Unknown')
            mention_details.append({
                'username': author_username,
                'tweet_id': tweet.id,
                'text': tweet.text,
                'created_at': tweet.created_at
            })

    return mention_details

username = 'MINDofPepe'
mentions = extract_mentions(bearer_token, username)

for mention in mentions:
    print(f"Username: {mention['username']}")
    print(f"Tweet ID: {mention['tweet_id']}")
    print(f"Text: {mention['text']}")
    print(f"Created At: {mention['created_at']}")
    print('-' * 40)


Username: abdul_hame22207
Tweet ID: 1916890348078465293
Text: @MINDofPepe Pepe is king
Created At: 2025-04-28 16:20:47+00:00
----------------------------------------
Username: lekside_4444
Tweet ID: 1916890295062462886
Text: @Emycryptolord @MINDofPepe God will deliver us from evil, army I know it’s weird to run around strangers to ask for a follow back but I am not asking you to follow me back just to boost my follower numbers but for an important reason please do follow back when you see this @Emycryptolord
Created At: 2025-04-28 16:20:35+00:00
----------------------------------------
Username: NadeetharangaSL
Tweet ID: 1916890230650552729
Text: @MINDofPepe pepe for pres
Created At: 2025-04-28 16:20:19+00:00
----------------------------------------
Username: rafsanJani93238
Tweet ID: 1916890229169946947
Text: @MINDofPepe Pepe for king
Created At: 2025-04-28 16:20:19+00:00
----------------------------------------
Username: Priyanka202404
Tweet ID: 1916890226733244669
Text: @MINDofPepe 

# Comparison

In [34]:
def filter_replies_by_usernames(replies, target_usernames):
    filtered_replies = []

    for reply in replies:
        if reply['username'] in target_usernames:
            filtered_replies.append({
                'tweet_id': reply['tweet_id'],
                'username': reply['username'],
                'created_at': reply['created_at'],
                'text': reply['text']
            })

    return filtered_replies


filtered_replies = filter_replies_by_usernames(replies, usernames)

for reply in filtered_replies:
    print(f"Reply ID: {reply['tweet_id']}, @{reply['username']} at {reply['created_at']}: {reply['text']}")
